In [14]:
pip install tensorflow


In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import shutil
import zipfile
import requests

In [16]:
dataset_url = "http://download.tensorflow.org/example_images/flower_photos.tgz"
dataset_path = tf.keras.utils.get_file('flower_photos.tgz', origin=dataset_url, extract=True)
base_dir = os.path.join(os.path.dirname(dataset_path), 'flower_photos')


228813984/228813984 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [17]:
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

In [18]:
# Create a train/validation split
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(val_dir):
    os.makedirs(val_dir)

In [19]:
class_names = os.listdir(base_dir)


In [20]:
for class_name in class_names:
    class_path = os.path.join(base_dir, class_name)
    # Check if class_path is a directory before trying to list its contents
    if os.path.isdir(class_path):
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)
        files = os.listdir(class_path)

In [21]:
# Split files into training and validation sets
train_files = files[:int(0.8 * len(files))]
val_files = files[int(0.8 * len(files)):]

In [22]:
for file in train_files:
        shutil.move(os.path.join(class_path, file), os.path.join(train_dir, class_name, file))

In [23]:
 for file in val_files:
        shutil.move(os.path.join(class_path, file), os.path.join(val_dir, class_name, file))


In [24]:
# Data preprocessing
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
val_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [25]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 718 images belonging to 7 classes.
Found 180 images belonging to 7 classes.


In [ ]:
# Load the base model (e.g., VGG16)
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Freeze the base model layers
base_model.trainable = False


In [ ]:
# Create a new model on top
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_names), activation='softmax')
])


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Create a new model on top
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(7, activation='softmax') # Change len(class_names) to 7
])

In [ ]:
# Fine-tune the model
base_model.trainable = True
# Unfreeze the last few layers
for layer in base_model.layers[-4:]:
    layer.trainable = True

In [ ]:
# Recompile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Continue training
history_finetune = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=10
)

Epoch 1/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 255ms/step - accuracy: 1.0000 - loss: 1.5450e-04 - val_accuracy: 1.0000 - val_loss: 2.0042e-06
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 1.0000 - loss: 7.8289e-05 - val_accuracy: 1.0000 - val_loss: 1.4663e-06
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 249ms/step - accuracy: 1.0000 - loss: 6.4820e-05 - val_accuracy: 1.0000 - val_loss: 1.5765e-06
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 5.0054e-05 - val_accuracy: 1.0000 - val_loss: 1.0073e-06
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 250ms/step - accuracy: 1.0000 - loss: 7.5463e-05 - val_accuracy: 1.0000 - val_loss: 1.1369e-06
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 3.1203e-05 - val_accuracy: 1.0000 - val_loss: 1.0669e-06
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 249ms/step - accuracy: 1.0000 - loss: 4.1274e-05 - val_accuracy: 1.0000 - val_loss: 9.7676e-07
Epoch 8/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - 

In [ ]:
loss, accuracy = model.evaluate(val_generator)
print(f'Validation accuracy: {accuracy:.2f}')

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - accuracy: 1.0000 - loss: 6.9422e-07
Validation accuracy: 1.00


In [9]:
def plot_accuracy(history):
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()
